In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm
from numpy.linalg import solve
from scipy.stats import f
from io import StringIO
# print(sm.__version__)

# Imports and plotting setup
%matplotlib inline

# Ignore all warnings
warnings.filterwarnings('ignore')

# 0) Import Data and Clean + Define Functions

In [ ]:
###################################################################################################
###################################################################################################
base_dir = os.getcwd()  # Current working directory

# Construct full paths using only the filenames
excel_path = os.path.join(base_dir, "start_end_fem21003.xlsx")
csv_factors_path = os.path.join(base_dir, "jkpfactors_US_all.csv")
portfolio_path = os.path.join(base_dir, "25_Portfolios_ME_OP_5x5.csv")
ff_factors_path = os.path.join(base_dir, "F-F_Research_Data_5_Factors_2x3.csv")

# Load the Excel file with start and end dates
excel_names = pd.read_excel(excel_path)

# Load CSV with Fama French factors
csv_data = pd.read_csv(csv_factors_path)

# Load portfolio data (skip metadata rows)
Double_sorted_portfolio = pd.read_csv(
    portfolio_path,
    skiprows=22,  # Skip metadata
    delim_whitespace=False,  # Use whitespace as separator
)

# Load Fama-French factors
factors = pd.read_csv(ff_factors_path, header=2)  # use this row as the column names

###################################################################################################
## DATES ##
###################################################################################################

# Get our rows of interest
excel_names.iloc[:, 1] = excel_names.iloc[:, 1].astype(str).str.strip()

row_yan = excel_names[excel_names.iloc[:, 1] == "Yan van"]
start_date = row_yan.iloc[0, 4]
end_date = row_yan.iloc[0, 5]

# Convert to YYYYMM format
start_yyyymm = pd.to_datetime(start_date).strftime("%Y%m")
end_yyyymm = pd.to_datetime(end_date).strftime("%Y%m")


###################################################################################################
## PORTFOLIO 5x5 ##
###################################################################################################
# Drop the last row
Double_sorted_portfolio = Double_sorted_portfolio.drop(
    Double_sorted_portfolio.index[-1]
)

# Rename first column to "Date"
Double_sorted_portfolio.rename(
    columns={Double_sorted_portfolio.columns[0]: "Date"}, inplace=True
)

# Keep only rows where Date is numeric and has more than 4 digits
Double_sorted_portfolio = Double_sorted_portfolio[
    pd.to_numeric(Double_sorted_portfolio["Date"], errors="coerce").notnull()
]

Double_sorted_portfolio = Double_sorted_portfolio.iloc[
    :769
] # This row number is the value where the first set of dates stops

# Convert Date to int safely
Double_sorted_portfolio["Date"] = Double_sorted_portfolio["Date"].astype(int)

# Convert start/end dates to integers
start_int = int(start_yyyymm)
end_int = int(end_yyyymm)

# Filter rows between start and end dates
filtered_portfolio = Double_sorted_portfolio[
    (Double_sorted_portfolio["Date"] >= start_int)
    & (Double_sorted_portfolio["Date"] <= end_int)
]

###################################################################################################
## FACTORS ##
###################################################################################################
# Clean column names
factors.columns = factors.columns.str.strip()

# Select first column + the factors
factors_subset = factors.iloc[
    :, [0, 1, 2, 3, 4, 6]
].copy()  # column 0 = Date, 1-4 = factors

# Rename the first column to "Date"
factors_subset.rename(columns={factors_subset.columns[0]: "Date"}, inplace=True)

factors_subset = factors_subset[factors_subset["Date"].astype(str).str.len() > 4]

filtered_factors = factors_subset[
    (factors_subset["Date"] >= start_yyyymm) & (factors_subset["Date"] <= end_yyyymm)
]

# Convert portfolio columns to numeric
portfolio_values = filtered_portfolio.drop(columns="Date").apply(
    pd.to_numeric, errors="coerce"
)

# Convert RF to numeric
filtered_factors["RF"] = pd.to_numeric(filtered_factors["RF"], errors="coerce")

# Drop any rows with NaNs in either portfolio or RF
valid_idx = portfolio_values.dropna().index.intersection(
    filtered_factors.dropna().index
)
portfolio_values = portfolio_values.loc[valid_idx]
rf_values = filtered_factors.loc[valid_idx, "RF"]

## Functions

In [ ]:
###################################################################################################
# Functions
###################################################################################################
def pca(X, gamma=0.00):
    """PCA / RP-PCA implementation"""
    X = np.asarray(X, dtype=float)
    N = len(X)  # time periods
    # Covariance component
    XX = X.T.dot(X) / float(N - 1)
    # Mean returns
    EX = X.mean(axis=0)
    X_bar = np.outer(EX, EX)
    sigma = XX + gamma * X_bar

    # Eigen-decomposition
    eigvals, eigvecs = np.linalg.eig(sigma)
    eigvals = np.real_if_close(eigvals)
    eigvecs = np.real_if_close(eigvecs)

    # Sort in descending order
    idx = np.argsort(eigvals)[::-1]
    eigvals_sorted = eigvals[idx]
    eigvecs_sorted = eigvecs[:, idx]

    relative_eigvals = eigvals_sorted / eigvals_sorted.sum()
    return eigvals_sorted, eigvecs_sorted, relative_eigvals

def run_rp_pca(X, gamma, k):
    eigvals, eigvecs, _ = pca(X, gamma=gamma)
    return eigvecs[:, :k]        

def estimate_betas(y, F):
    """Regress each column of y on F (with constant) and return betas (n×k)"""
    betas = []
    for i in range(y.shape[1]):
        model = sm.OLS(y[:, i], sm.add_constant(F)).fit()
        betas.append(model.params[1:])   
    return np.vstack(betas)              

def rolling_window_oos(ret, window=240, gamma=-1, k=1):
    """
    Rolling OOS estimation for PCA / RP-PCA:
      - Uses 240-month rolling windows
      - Returns prediction errors and tangency portfolio returns
    """
    T, N = ret.shape
    oos_errors, oos_preds, tangency_returns = [], [], []

    for t in range(window, T-1):
        train = ret[t-window:t, :]

        # 1. Estimate PCA / RP-PCA factors for this window
        eigvecs = run_rp_pca(train, gamma, k) 
        factors_t = train @ eigvecs          

        # 2. Estimate betas 
        betas_t = estimate_betas(train, factors_t)

        # Tangency portfolio weights 
        mu_F = factors_t.mean(axis=0).reshape(-1, 1)
        Sigma_F = np.cov(factors_t.T)
        
        if np.ndim(Sigma_F) == 0: # Ensure Sigma_F is 2D (1x1) when k = 1
            Sigma_F = np.array([[Sigma_F]])

        ones = np.ones((k, 1))
        Sigma_inv = np.linalg.inv(Sigma_F)
        w_tang = Sigma_inv @ mu_F
        w_tang = w_tang / (ones.T @ Sigma_inv @ mu_F)
        # 3. R_{t+1} * Gamma_t 
        next_f = ret[t+1, :] @ eigvecs       

        # 4. Predict next month
        rhat = betas_t @ next_f
        err  = ret[t+1, :] - rhat

        # 5. Tangency portfolio realized return
        r_star = float(w_tang.T @ next_f.reshape(-1,1))

        # Store
        oos_errors.append(err)
        oos_preds.append(rhat)
        tangency_returns.append(r_star)

    return np.array(oos_errors), np.array(oos_preds), np.array(tangency_returns)

def evaluate_oos(errors, preds, r_star):
    """
    Compute OOS evaluation metrics:
      RMS alpha, ARIV, and Sharpe ratio of tangency portfolio
    """
    # Pricing metrics
    rms_alpha = np.sqrt(np.mean(np.mean(errors, axis=0)**2))
    var_resid = np.var(errors, axis=0, ddof=1)
    var_total = np.var(preds + errors, axis=0, ddof=1)
    ariv = np.mean(var_resid / var_total)

    # Tangency Sharpe ratio
    mu_star = np.mean(r_star)
    sigma_star = np.std(r_star, ddof=1)
    sharpe_star = mu_star / sigma_star

    return float(rms_alpha), float(ariv), float(sharpe_star)

def rolling_window_oos_given_factors(ret, factors_df, window=240):
    T, N = ret.shape
    factors_arr = factors_df.to_numpy()
    k = factors_arr.shape[1]
    
    oos_errors, oos_preds, tangency_returns = [], [], []
    
    for t in range(window, T-1):
        # 1. Take last 240 months of data
        R_train = ret[t-window:t, :]
        F_train = factors_arr[t-window:t, :]
        
        # 2. Estimate betas (OLS)
        betas = []
        for i in range(N):
            model = sm.OLS(R_train[:, i], sm.add_constant(F_train)).fit()
            betas.append(model.params[1:])
        betas = np.vstack(betas)
        
        # 3. Tangency weights (using factor moments)
        mu_F = F_train.mean(axis=0).reshape(-1,1)
        Sigma_F = np.cov(F_train.T)
        
        # Ensure Sigma_F is 2D even for single factor
        if np.ndim(Sigma_F) == 0:
            Sigma_F = np.array([[Sigma_F]])
        
        Sigma_inv = np.linalg.inv(Sigma_F)
        ones = np.ones((k,1))
        w_tang = Sigma_inv @ mu_F
        w_tang = w_tang / (ones.T @ Sigma_inv @ mu_F)
        
        # 4. Next month's factors and predictions
        F_next = factors_arr[t+1, :]
        rhat = betas @ F_next
        err = ret[t+1, :] - rhat
        r_star = float(w_tang.T @ F_next.reshape(-1,1))
        
        oos_errors.append(err)
        oos_preds.append(rhat)
        tangency_returns.append(r_star)
    
    return np.array(oos_errors), np.array(oos_preds), np.array(tangency_returns)

def choose_k(eigvals, threshold=0.8):
    cum = np.cumsum(eigvals) / np.sum(eigvals)
    return np.argmax(cum >= threshold) + 1

def evaluate_oos_alphas(errors, portfolio_names):
    """
    Compute OOS alphas per portfolio and return as a DataFrame.
    """
    errors = np.asarray(errors)
    oos_alpha = np.mean(errors, axis=0)  # average across time

    df_alpha = pd.DataFrame(
        oos_alpha,
        index=portfolio_names,
        columns=["alpha_OOS"]
    )
    return df_alpha

# Q1)

In [ ]:
###################################################################################################
## Q1
###################################################################################################
returns_factors_canvas = csv_data.copy()

###################################################################################################
## RF ##
###################################################################################################
rf_monthly = filtered_factors["RF"] / 100  

###################################################################################################
## PORTFOLIOS ##
###################################################################################################
## Creating the PORTFOLIO excess returns ##
portfolio_scaled = filtered_portfolio.copy()

# Convert all columns except 'Date' to numeric and divide by 100
portfolio_scaled.iloc[:, 1:] = (
    portfolio_scaled.iloc[:, 1:].apply(pd.to_numeric, errors="coerce").div(100)
)

# Create excess returns by subtracting rf_monthly from each return column
portfolio_excess = portfolio_scaled.copy()
for col in portfolio_scaled.columns:
    if col != "Date":  
        portfolio_excess[col] = portfolio_scaled[col] - rf_monthly


# Define the excess returns mat for Q3
excess_portfolio_returns_q3 = portfolio_excess.copy()

r_f_fixed = 0.0025

# Mean Excess Returns / COV MATRIX OF EXCESS RETURNS
mean_excess_returns = portfolio_excess.drop("Date", axis=1).mean()
sigma_excess_returns = portfolio_excess.drop("Date", axis=1).cov()
sigma_inv_excess_returns = np.linalg.inv(sigma_excess_returns)
ones = np.ones(len(mean_excess_returns)).reshape(-1, 1)  # Make it a column vector
mean_returns = mean_excess_returns + r_f_fixed
mean_returns = mean_returns.values.reshape(-1, 1)  # Make it a column vector


###################################################################################################
## CONSTANTS ##
###################################################################################################
A = (mean_returns.T @ sigma_inv_excess_returns @ mean_returns).item()
B = (mean_returns.T @ sigma_inv_excess_returns @ ones).item()
C = (ones.T @ sigma_inv_excess_returns @ ones).item()

###################################################################################################
## GMV ##
###################################################################################################
pi_gmv = ((1 / C) * (sigma_inv_excess_returns @ ones)).flatten()
mean_gmv_portfolio = B / C
variance_gmv_portfolio = 1 / C
std_dev_gmv_portfolio = np.sqrt(variance_gmv_portfolio)

###################################################################################################
# 1: Efficient frontier without risk-free asset
###################################################################################################
## B != 0 hence pi_mu solution ##
pi_mu = (1 / B) * (
    sigma_inv_excess_returns @ mean_returns
).flatten()  # Mean-variance portfolio
mus = np.linspace(0.0025, 0.03, 400)

variances = []
for mu_targ in mus:
    labda = (B * C * mu_targ - B**2) / (A * C - B**2)  # from lecture 3
    pi_mv = labda * pi_mu + (1 - labda) * pi_gmv
    var_mv = pi_mv.T @ sigma_excess_returns.values @ pi_mv
    variances.append(var_mv)

std_devs_no_rf = np.sqrt(variances)

###################################################################################################
# 2: Efficient frontier WITH risk-free asset
###################################################################################################
## TANGENCY PORTFOLIO ##
pi_tan = (
    (1 / (ones.T @ sigma_inv_excess_returns @ mean_excess_returns))
    * (sigma_inv_excess_returns @ mean_excess_returns)
).flatten()

mean_tan = (pi_tan @ mean_returns).flatten()[0]
variance_tan = pi_tan.T @ sigma_excess_returns @ pi_tan
std_dev_tan = np.sqrt(variance_tan)

# CAL
sharpe_tan = (mean_tan - r_f_fixed) / std_dev_tan
cal_x = np.linspace(0, std_devs_no_rf.max(), 200)
cal_y = r_f_fixed + sharpe_tan * cal_x

###################################################################################################
# 3: Factors (Rf + Mkt/SMB/RMW)
###################################################################################################
fac = filtered_factors.copy()
# Convert relevant columns to numeric and divide by 100
for col in ["Mkt-RF", "HML", "SMB", "RMW", "RF"]:
    fac[col] = pd.to_numeric(fac[col], errors="coerce") / 100.0

# 2) Compute means and std devs (already excess returns for factors)
mu_mkt, sd_mkt = fac["Mkt-RF"].mean(), fac["Mkt-RF"].std(ddof=1)
mu_smb, sd_smb = fac["SMB"].mean(), fac["SMB"].std(ddof=1)
mu_rmw, sd_rmw = fac["RMW"].mean(), fac["RMW"].std(ddof=1)

# 3) Slopes = Sharpe ratios (mean excess / st.dev)
slope_mkt = mu_mkt / sd_mkt
slope_smb = mu_smb / sd_smb
slope_rmw = mu_rmw / sd_rmw

# 4) Build CALs
sigma_max = (
    max(std_devs_no_rf.max(), std_dev_tan, sd_mkt, sd_smb, sd_rmw) * 1.3
)  
sig_grid = np.linspace(0, sigma_max, 200)

cal_mkt_y = r_f_fixed + slope_mkt * sig_grid
cal_smb_y = r_f_fixed + slope_smb * sig_grid
cal_rmw_y = r_f_fixed + slope_rmw * sig_grid


###################################################################################################
# FIGURE (Computation Only): Efficient frontier using 3 factors (Mkt-RF, SMB, RMW)
###################################################################################################
Xf = fac[["Mkt-RF", "SMB", "RMW"]].dropna()

# Compute mean excess returns and covariance matrix
mu_e_f = Xf.mean().values.reshape(-1, 1)  
Sigma_f = Xf.cov().values                 
Sigma_f_inv = np.linalg.inv(Sigma_f)
ones_f = np.ones((3, 1))

# Expected (not excess) returns for plotting
mu_f = mu_e_f + r_f_fixed

# Frontier constants for 3 factors
A_f = float(mu_f.T @ Sigma_f_inv @ mu_f)
B_f = float(mu_f.T @ Sigma_f_inv @ ones_f)
C_f = float(ones_f.T @ Sigma_f_inv @ ones_f)

# GMV portfolio (3 factors)
pi_gmv_f = ((1 / C_f) * (Sigma_f_inv @ ones_f)).flatten()
mu_gmv_f = B_f / C_f
var_gmv_f = 1 / C_f
sd_gmv_f = np.sqrt(var_gmv_f)

# Mean–variance portfolios (no risk-free)
pi_mu_f = (1 / B_f) * (Sigma_f_inv @ mu_f).flatten()

# μ-grid for the efficient frontier
mu_min_f = max(r_f_fixed, mu_f.min() * 0.6)
mu_max_f = float(mu_f.max() * 1.3)
mus_f = np.linspace(mu_min_f, mu_max_f, 300)

# Variances for the frontier
vars_f = []
for mu_targ in mus_f:
    lam_f = (B_f * C_f * mu_targ - B_f**2) / (A_f * C_f - B_f**2)
    pi_mv_f = lam_f * pi_mu_f + (1 - lam_f) * pi_gmv_f
    vars_f.append(pi_mv_f.T @ Sigma_f @ pi_mv_f)

sds_f = np.sqrt(np.array(vars_f))

# Tangency portfolio (with risk-free asset)
pi_tan_f = ((Sigma_f_inv @ mu_e_f) / (ones_f.T @ Sigma_f_inv @ mu_e_f)).flatten()
mu_tan_f = float(pi_tan_f @ mu_f.flatten())  # expected (includes rf)
var_tan_f = float(pi_tan_f.T @ Sigma_f @ pi_tan_f)
sd_tan_f = np.sqrt(var_tan_f)

# CAL for the 3-factor tangency
sharpe_f = (mu_tan_f - r_f_fixed) / sd_tan_f
cal3_x = np.linspace(0, max(sds_f.max(), sd_tan_f, std_devs_no_rf.max()) * 1.1, 200)
cal3_y = r_f_fixed + sharpe_f * cal3_x

# Percent versions (for plotting)
mus_f_pct = mus_f * 100
sds_f_pct = sds_f * 100
mu_tan_f_pct = mu_tan_f * 100
sd_tan_f_pct = sd_tan_f * 100
cal3_x_pct = cal3_x * 100
cal3_y_pct = cal3_y * 100


###################################################################################################
# Q1 Figure: Efficient Frontiers — 25 Portfolios vs 3-Factor Universe
###################################################################################################

plt.figure(figsize=(10, 6))

# Convert to percentages
mus_pct = mus * 100
std_devs_no_rf_pct = std_devs_no_rf * 100
cal_x_pct = cal_x * 100
cal_y_pct = cal_y * 100
mean_tan_pct = mean_tan * 100
std_dev_tan_pct = std_dev_tan * 100
mean_gmv_portfolio_pct = mean_gmv_portfolio * 100
std_dev_gmv_portfolio_pct = std_dev_gmv_portfolio * 100

# (1) Efficient Frontier (25 portfolios, no RF)
plt.plot(std_devs_no_rf_pct, mus_pct, label="Efficient Frontier (25 Portfolios)")

# (2) CAL for 25 portfolios
plt.plot(cal_x_pct, cal_y_pct, linestyle="--", label="CAL (25 Portfolios)")
plt.scatter(std_dev_tan_pct, mean_tan_pct, marker="*", s=150, label="Tangency (25)")
plt.scatter(std_dev_gmv_portfolio_pct, mean_gmv_portfolio_pct, marker="o", s=90, label="GMV (25)")

# (3) CAL for 3-factor tangency
plt.plot(cal3_x_pct, cal3_y_pct, linestyle=":", label="CAL (3 Factors: Mkt, SMB, RMW)")
plt.scatter(sd_tan_f_pct, mu_tan_f_pct, marker="D", s=120, label="Tangency (3 Factors)")

# (4) 3-factor efficient frontier (no RF)
plt.plot(sds_f_pct, mus_f_pct, label="Efficient Frontier (3 Factors)")

# (5) Individual portfolios
portfolio_means_pct = portfolio_excess.drop("Date", axis=1).mean() * 100
portfolio_stds_pct = portfolio_excess.drop("Date", axis=1).std(ddof=1) * 100
plt.scatter(portfolio_stds_pct, portfolio_means_pct, marker="x", s=60, label="25 Portfolios")

# Cosmetics
plt.xlabel("Volatility (%)")
plt.ylabel("Expected Return (%)")
plt.title("Efficient Frontiers: 25 Portfolios (with & without RF) vs 3-Factor Universe")
plt.legend()
plt.grid(True)
plt.ylim(0.25, 3.0)
plt.xlim(0, 8.0)
plt.show()


###################################################################################################
# SUMMARY TABLE: GMV, Tangency (25 portfolios), and Factor Tangencies (Mkt, SMB, RMW)
###################################################################################################

# GMV
gmv_mean_pct = mean_gmv_portfolio * 100
gmv_vol_pct = std_dev_gmv_portfolio * 100
gmv_sharpe = (mean_gmv_portfolio - r_f_fixed) / std_dev_gmv_portfolio

# Tangency (25 portfolios)
tan_mean_pct = mean_tan * 100
tan_vol_pct = std_dev_tan * 100
tan_sharpe = (mean_tan - r_f_fixed) / std_dev_tan

# Market (as tangency)
mkt_mean_pct = (r_f_fixed + mu_mkt) * 100
mkt_vol_pct = sd_mkt * 100
mkt_sharpe = mu_mkt / sd_mkt

# SMB
smb_mean_pct = (r_f_fixed + mu_smb) * 100
smb_vol_pct = sd_smb * 100
smb_sharpe = mu_smb / sd_smb

# RMW (Profitability)
rmw_mean_pct = (r_f_fixed + mu_rmw) * 100
rmw_vol_pct = sd_rmw * 100
rmw_sharpe = mu_rmw / sd_rmw

summary = pd.DataFrame(
    {
        "Expected Return (%)": [
            gmv_mean_pct,
            tan_mean_pct,
            mkt_mean_pct,
            smb_mean_pct,
            rmw_mean_pct,
        ],
        "Volatility (%)": [
            gmv_vol_pct,
            tan_vol_pct,
            mkt_vol_pct,
            smb_vol_pct,
            rmw_vol_pct,
        ],
        "Sharpe Ratio": [gmv_sharpe, tan_sharpe, mkt_sharpe, smb_sharpe, rmw_sharpe],
    },
    index=[
        "GMV (25 Portfolios)",
        "Tangency (25 Portfolios)",
        "Market (Mkt-RF)",
        "SMB",
        "RMW (Operating Profitability)",
    ],
)

# Round for reporting
summary = summary.round(2)

# Tangency (3 Factors: Mkt-RF, SMB, RMW)
tan3f_mean_pct = mu_tan_f * 100
tan3f_vol_pct  = sd_tan_f * 100
tan3f_sharpe   = sharpe_f

# Add to summary DataFrame
summary.loc["Tangency (3 Factors)"] = [tan3f_mean_pct, tan3f_vol_pct, tan3f_sharpe]

# Round again for reporting
summary = summary.round(2)


print("\nSummary Table:")
print(summary)


# Q2) 


In [ ]:
###################################################################################################
## Portfolio Returns ##
R = portfolio_excess.drop(columns=['Date'])   # excess returns of 25 portfolios
R = R.apply(pd.to_numeric, errors='coerce')
T, n = R.shape
k = 1  # number of factors (CAPM)

fac_q2 = fac.copy() 
fac_q2_no_date = fac_q2.drop(columns=['Date'])

#######################################################################################################
### CAPM ###
#######################################################################################################
## STEP 1: REGRESSIONS ##
#######################################################################################################
factors_capm = fac_q2_no_date[['Mkt-RF']]
alphas_capm, ses_capm, resid_capm = [], [], []

### CAPM regressions ###
X_capm = sm.add_constant(factors_capm)  # add intercept
for col in R.columns:
    y = R[col].values
    model = sm.OLS(y, X_capm).fit()
    alphas_capm.append(model.params[0])     # alpha 
    ses_capm.append(model.bse[0])           # std error of alpha
    resid_capm.append(model.resid)          # residuals

# To arrays
alphas_capm = np.array(alphas_capm)       # 25 x 1
ses_capm    = np.array(ses_capm)          # 25 x 1
resid_capm  = np.array(resid_capm).T      # T x 25

sigma_capm_ret = resid_capm @ resid_capm.T / T  # T x T

# Factor mean and Covariance mu_f and sigma_f
F = factors_capm.values
mu_f = F.mean(axis=0).reshape(-1,1)  # k x 1
F_centered = F - mu_f
Sigma_f_tilde = (F_centered.T @ F_centered) / T # k x k

# Prepare alphas and residual covariance
alpha = np.asarray(alphas_capm).reshape(-1, 1)   # n x 1
Sigma_e_tilde =  (1/T) * (resid_capm.T @ resid_capm)  # 25 x 25    

# Compute GRS statistic
s = 1.0 + (mu_f @ solve(Sigma_f_tilde, mu_f.T))[0, 0]
num = (alpha.T @ solve(Sigma_e_tilde, alpha))[0, 0]

df1 = n
df2 = T - n - k 
GRS = ((T - n - k) / n) * (1.0 / s) * num
pval = 1.0 - f.cdf(GRS, df1, df2)

print(f"GRS = {GRS:.4f},  p-value = {pval:.4g}, df=({df1},{df2})")

# Compute RMS alpha
RMS_alpha = np.sqrt(np.mean(alphas_capm.flatten()**2))

# Compute ARIV
var_resid = np.var(resid_capm, axis=0, ddof=0)       # variance of residuals per portfolio
var_total = np.var(R, axis=0, ddof=0)                # variance of returns per portfolio
ARIV = np.mean(var_resid / var_total)

# Build table
results_df = pd.DataFrame({
    "Portfolio": R.columns,
    "Alpha (%)": alphas_capm.flatten() * 100,
    "SE Alpha (%)": ses_capm.flatten() * 100
})

# Add overall statistics at bottom
summary_df = pd.DataFrame({
    "Portfolio": ["RMS Alpha", "ARIV", "GRS", "p-value"],
    "Alpha (%)": [RMS_alpha, ARIV, GRS, pval],
    "SE Alpha (%)": ["", "", "", ""]
})

# Final table
final_table = pd.concat([results_df, summary_df], ignore_index=True)

print(final_table)

###################################################################################################
### 3-Factor regressions ###
###################################################################################################
# Factors: Mkt-RF, SMB, RMW
factors_3f = fac_q2_no_date[["Mkt-RF", "SMB", "RMW"]]
alphas_3f, ses_3f, resid_3f = [], [], []

X_3f = sm.add_constant(factors_3f)
for col in R.columns:
    y = R[col].values
    model = sm.OLS(y, X_3f).fit()
    alphas_3f.append(model.params[0])  # intercept = alpha
    ses_3f.append(model.bse[0])        # std error of alpha
    resid_3f.append(model.resid)       # residuals

# To arrays 
alphas_3f = np.array(alphas_3f).reshape(-1,1)   # n x 1 column vector
ses_3f    = np.array(ses_3f).reshape(-1,1)
resid_3f  = np.array(resid_3f).T                # T x n

# Factor mean and Covariance mu_f and sigma_f
F3 = factors_3f.values
mu_f3 = F3.mean(axis=0).reshape(-1,1)                # k x 1
F3_centered = F3 - mu_f3.T
Sigma_f3_tilde = (F3_centered.T @ F3_centered) / T   # k x k

# Residual covariance
Sigma_e3_tilde = (1/T) * (resid_3f.T @ resid_3f)

# Prepare GRS statistic components
mu_f3_inv = np.linalg.inv(Sigma_f3_tilde) @ mu_f3
s3 = 1.0 + float(mu_f3.T @ mu_f3_inv)
alpha_inv = np.linalg.inv(Sigma_e3_tilde) @ alphas_3f
num3 = float(alphas_3f.T @ alpha_inv)

# GRS test statistic
df1, df2 = n, T - n - 3
GRS_3f = ((T - n - 3) / n) * (num3 / s3)
pval_3f = 1.0 - f.cdf(GRS_3f, df1, df2)

print(f"GRS (3-factor) = {GRS_3f:.4f},  p-value = {pval_3f:.4g}, df=({df1},{df2})")

###################################################################################################
### Summary stats: RMS Alpha & ARIV ###
###################################################################################################
RMS_alpha_3f = np.sqrt(np.mean(alphas_3f.flatten()**2))

var_resid_3f = np.var(resid_3f, axis=0, ddof=0)
var_total_3f = np.var(R, axis=0, ddof=0)
ARIV_3f = np.mean(var_resid_3f / var_total_3f)

print(f"RMS Alpha (3f): {RMS_alpha_3f:.6f}")  # keep in decimals
print(f"ARIV (3f): {ARIV_3f:.6f}")            # keep in decimals

###################################################################################################
# TABLE #
###################################################################################################
# Build table for 3-factor model
results_df_3f = pd.DataFrame({
    "Portfolio": R.columns,
    "Alpha": alphas_3f.flatten(),        # decimals
    "SE Alpha": ses_3f.flatten()         # decimals
})

# Add overall statistics at bottom
summary_df_3f = pd.DataFrame({
    "Portfolio": ["RMS Alpha", "ARIV", "GRS", "p-value"],
    "Alpha": [RMS_alpha_3f, ARIV_3f, GRS_3f, pval_3f],
    "SE Alpha": ["", "", "", ""]
})

# Final table
final_table_3f = pd.concat([results_df_3f, summary_df_3f], ignore_index=True)
print(final_table_3f)


###################################################################################################
### Sharpe Ratios ###
###################################################################################################
# CAPM: Market Sharpe ratio 
# Mkt-RF column is already excess return of market
market_excess = fac_q2_no_date["Mkt-RF"].values 
mu_m = np.mean(market_excess)                          # mean excess return
sigma_m = np.std(market_excess, ddof=0)                # std dev of excess return
Sharpe_market = mu_m / sigma_m

print(f"Sharpe ratio (Market, CAPM): {Sharpe_market:.4f}")

# 3-Factor Model: Efficient portfolio Sharpe ratio 
mu_f3 = F3.mean(axis=0).reshape(-1,1)                # k x 1 mean of factors
Sigma_f3 = np.cov(F3, rowvar=False, ddof=0)          # k x k covariance matrix of factors
Sharpe_3f = np.sqrt(float(mu_f3.T @ np.linalg.inv(Sigma_f3) @ mu_f3))

# CAPM efficient Sharpe (includes alphas) 
Sharpe_capm_sq = float(mu_f.T @ np.linalg.inv(Sigma_f_tilde) @ mu_f) + \
                 float(alphas_capm.T @ np.linalg.inv(Sigma_e_tilde) @ alphas_capm)
Sharpe_capm = np.sqrt(Sharpe_capm_sq)
print(f"CAPM Sharpe ratio including alphas: {Sharpe_capm:.4f}")

# 3-Factor efficient Sharpe (includes alphas)
Sharpe_3f_sq = float(mu_f3.T @ np.linalg.inv(Sigma_f3_tilde) @ mu_f3) + \
               float(alphas_3f.T @ np.linalg.inv(Sigma_e3_tilde) @ alphas_3f)
Sharpe_3f_full = np.sqrt(Sharpe_3f_sq)
print(f" 3-factor Sharpe ratio including alphas: {Sharpe_3f_full:.4f}")


# Q3) 

In [ ]:
###################################################################################################
### DATA CLEAN ###
###################################################################################################
excess_portfolio_returns_q3['Date'] = pd.to_datetime(excess_portfolio_returns_q3['Date'])

date = excess_portfolio_returns_q3['Date']
ret_df = excess_portfolio_returns_q3.drop(columns=['Date'])
labels = ret_df.columns.tolist()
ret = ret_df.to_numpy(dtype=float)

###################################################################################################
### SET GAMMA AND FACTOR LIST ###
###################################################################################################
gamma_list = [-1, 0, 20]   # -1 and 0 = PCA, 20 = RP-PCA
num_factors_list = [1, 3]  # single factor and three factors

###################################################################################################
### RUN PCA FOR DIFFERENT GAMMA AND K ###
###################################################################################################
results = {}

for gamma in gamma_list:
    for k in num_factors_list:
        PCeigval, PCeigvec, PCrelative_eigval = pca(ret, gamma=gamma)

        results[(gamma, k)] = {
            "eigvals": PCeigval[:k],
            "eigvecs": PCeigvec[:, :k],
            "rel_explained": PCrelative_eigval[:k]
        }
        
        print(f"Gamma = {gamma}, k = {k}")
        print("Eigenvalues:", PCeigval[:k])
        print("Relative explained variance:", PCrelative_eigval[:k])
        print("-" * 50)

##########################################################################
### RUN FACTOR REGRESSIONS AND COLLECT RESULTS ###
##########################################################################
summary_rows = []  # model-level summary
alpha_tables = []  # per-portfolio alpha/SE tables

for (gamma, k), res in results.items():
    eigvecs = res["eigvecs"]
    factors = ret @ eigvecs
    res["factors"] = factors

    alphas, alphas_se, betas, r2s, resid_vars, total_vars = [], [], [], [], [], []

    for i in range(ret.shape[1]):  # loop portfolios
        y = ret[:, i]
        X = sm.add_constant(factors)
        model = sm.OLS(y, X).fit()

        alphas.append(model.params[0])
        alphas_se.append(model.bse[0])
        betas.append(model.params[1:])
        r2s.append(model.rsquared)
        resid_vars.append(np.var(model.resid, ddof=1))
        total_vars.append(np.var(y, ddof=1))

    # Save arrays
    res["alphas"] = np.array(alphas)
    res["alphas_se"] = np.array(alphas_se)
    res["betas"] = np.vstack(betas)
    res["r2"] = np.array(r2s)

    # Model-level summary
    rmse_alpha = np.sqrt(np.mean(res["alphas"]**2))
    ariv = np.mean(np.array(resid_vars) / np.array(total_vars))
    res["alpha_rms"] = rmse_alpha
    res["ariv"] = ariv

    summary_rows.append({
        "Gamma": gamma,
        "Model": "PCA" if gamma == -1 else "RP-PCA",
        "Factors": k,
        "RMSE alpha": rmse_alpha,
        "ARIV": ariv
    })

    # Portfolio-level alpha ± SE
    model_name = f"{'PCA' if gamma == -1 else 'RP-PCA'} (gamma={gamma}, k={k})"
    df_model = pd.DataFrame({
        "Portfolio": labels,
        model_name: [f"{a:.4f} ({se:.4f})" for a, se in zip(res["alphas"], res["alphas_se"])]
    })
    alpha_tables.append(df_model)

##########################################################################
### BUILD SUMMARY TABLES ###
##########################################################################
summary_df = pd.DataFrame(summary_rows)
print("\n=== Model-level Summary ===")
print(summary_df)

# Merge all alpha tables
alpha_table = alpha_tables[0]
for df_model in alpha_tables[1:]:
    alpha_table = alpha_table.merge(df_model, on="Portfolio")

print("\n=== Portfolio-level Alphas ± SE ===")
print(alpha_table)  

# Q4) 


In [ ]:
###################################################################################################
## OOS: Loop over all (γ, k) combinations and evaluate 
###################################################################################################
window = 240
gamma_list = [-1, 0, 20]
k_list = [1, 3]

oos_summary = []

for gamma in gamma_list:
    for k in k_list:
        print(f"Running OOS for gamma={gamma}, k={k} ...")
        errors, preds, r_star = rolling_window_oos(ret, window=window, gamma=gamma, k=k)
        rms_alpha, ariv, sharpe = evaluate_oos(errors, preds, r_star)
        oos_summary.append({
            "Gamma": gamma,
            "Model": "PCA" if gamma==-1 else "RP-PCA",
            "Factors": k,
            "OOS RMSα": rms_alpha,
            "OOS ARIV": ariv,
            "OOS SR": sharpe
        })

oos_results_df = pd.DataFrame(oos_summary)
# display(oos_results_df.round(4))


errors_capm, preds_capm, r_star_capm = rolling_window_oos_given_factors(ret, fac[["Mkt-RF"]])
rms_capm, ariv_capm, sharpe_capm = evaluate_oos(errors_capm, preds_capm, r_star_capm)

errors_3f, preds_3f, r_star_3f = rolling_window_oos_given_factors(ret, fac[["Mkt-RF","SMB","RMW"]])
rms_3f, ariv_3f, sharpe_3f = evaluate_oos(errors_3f, preds_3f, r_star_3f)

oos_capm_3f = pd.DataFrame({
    "Model": ["CAPM (Mkt-RF)", "3-Factor (Mkt, SMB, RMW)"],
    "OOS RMSα": [rms_capm, rms_3f],
    "OOS ARIV": [ariv_capm, ariv_3f],
    "OOS SR": [sharpe_capm, sharpe_3f]
}).round(4)

# display(oos_capm_3f)

# Combine PCA/RP-PCA results with CAPM and 3-Factor 
final_oos_table = pd.concat([
    oos_results_df[["Model", "Gamma", "Factors", "OOS RMSα", "OOS ARIV", "OOS SR"]],
    pd.DataFrame({
        "Model": ["CAPM", "Fama–French 3-Factor"],
        "Gamma": [np.nan, np.nan],
        "Factors": [1, 3],
        "OOS RMSα": [rms_capm, rms_3f],
        "OOS ARIV": [ariv_capm, ariv_3f],
        "OOS SR": [sharpe_capm, sharpe_3f]
    })
], ignore_index=True)

# Round 
final_oos_table = final_oos_table.round(4)
display(final_oos_table)

###################################################################################################
## OOS: Alphas
###################################################################################################
# Fetch the porftolio names
portfolio_names = Double_sorted_portfolio.columns[1:]

alpha_table = pd.DataFrame(index=portfolio_names)

# PCA / RP-PCA Models
for gamma in gamma_list:
    for k in k_list:
        model_name = f"{f'PCA (gamma = {gamma}' if gamma==-1 else f'RP-PCA(gamma={gamma}'}, k= {k})"
        errors, preds, r_star = rolling_window_oos(ret, window=240, gamma=gamma, k=k)
        alpha_table[model_name] = evaluate_oos_alphas(errors, portfolio_names)["alpha_OOS"]

# CAPM 
errors_capm, _, _ = rolling_window_oos_given_factors(ret, fac[["Mkt-RF"]])
alpha_table["CAPM"] = evaluate_oos_alphas(errors_capm, portfolio_names)["alpha_OOS"]

# 3-Factor 
errors_3f, _, _ = rolling_window_oos_given_factors(ret, fac[["Mkt-RF","SMB","RMW"]])
alpha_table["3-Factor"] = evaluate_oos_alphas(errors_3f, portfolio_names)["alpha_OOS"]

alpha_table = alpha_table.round(4)

last_three_transposed = final_oos_table.iloc[:, -3:].T

# Set the column names of the transposed part to match alpha_table columns
last_three_transposed.columns = alpha_table.columns

# Use original column names as row names
last_three_transposed.index.name = None  # Optional: remove index name
last_three_transposed.index = final_oos_table.columns[-3:]

# Append to alpha_table
merged_table = pd.concat([alpha_table, last_three_transposed])

# Display the result
display(merged_table)



# Q5) 

In [ ]:
###################################################################################################
# Q5: Factr Zoo OOS
###################################################################################################
# Load and clean Jensen factors
jkf = csv_data.copy()
jkf.columns = jkf.columns.str.strip()

# Ensure proper date parsing
jkf["Date"] = pd.to_datetime(jkf["date"], errors="coerce")
jkf["YYYYMM"] = jkf["Date"].dt.strftime("%Y%m").astype(float)

# Filter
jkf = jkf[(jkf["YYYYMM"] >= float(start_yyyymm)) & (jkf["YYYYMM"] <= float(end_yyyymm))].copy()

# Keep only numeric factor columns
jkf_factors = jkf.drop(columns=["date", "Date", "YYYYMM"]).apply(pd.to_numeric, errors="coerce")

# 25 double-sorted portfolios
ret_q5 = (
    filtered_portfolio
    .drop(columns=["Date"])
    .apply(pd.to_numeric, errors="coerce") / 100.0   # Convert % to decimal
)

# Align the two datasets (ensure same number of months)
min_T = min(len(ret_q5), len(jkf_factors))
ret_q5 = ret_q5.iloc[-min_T:, :].to_numpy(dtype=float)
factors153 = jkf_factors.iloc[-min_T:, :].to_numpy(dtype=float)
dates_aligned = jkf["Date"].iloc[-min_T:].reset_index(drop=True)


###################################################################################################
# Combine into a single matrix
###################################################################################################
combined_matrix = np.hstack([ret_q5, factors153])
print("Combined matrix shape:", combined_matrix.shape)

###################################################################################################
# PCA / RP-PCA eigenvalues and Scree Plot 
###################################################################################################

gammas = [-1, 0, 20]
eigvals_dict = {}
threshold = 0.9   # Treshold

plt.figure(figsize=(8,5))

for gamma in gammas:
    eigvals, eigvecs, rel_eigvals = pca(combined_matrix, gamma=gamma)
    eigvals_dict[gamma] = eigvals

    # Plot descending eigenvalues 
    plt.plot(
        range(1, len(eigvals)+1),
        eigvals / np.sum(eigvals),
        marker="o",
        label=f"γ = {gamma}"
    )

plt.title("Scree Plot – PCA vs RP-PCA (Descending Eigenvalues)")
plt.xlabel("Factor Number (sorted descending)")
plt.ylabel("Proportion of Total Variance")
plt.legend()
plt.grid(True)
plt.show()

for gamma in gammas:
    k_star = choose_k(eigvals_dict[gamma], threshold=threshold)
    print(f"γ = {gamma}: suggested k = {k_star} (90% variance explained)")

###################################################################################################
# Out-of-Sample Evaluation: PCA / RP-PCA vs Benchmarks
###################################################################################################

window = 240
k_values = {gamma: choose_k(eigvals_dict[gamma], threshold=threshold) for gamma in gammas}

results = []

###################################################################################################
# Rolling OOS for PCA / RP-PCA
###################################################################################################

for gamma_val in gammas:
    for k_val in [1, 3, k_values[gamma_val]]:
        print(f"Running OOS for γ={gamma_val}, k={k_val}")
        errors, preds, rstar = rolling_window_oos(
            combined_matrix, window=window, gamma=gamma_val, k=k_val
        )
        rms_alpha, ariv, sharpe = evaluate_oos(errors, preds, rstar)
        results.append({
            "Model": "RP-PCA" if gamma_val >= 0 else "PCA",
            "Gamma": gamma_val,
            "k": k_val,
            "RMS_alpha": rms_alpha,
            "ARIV": ariv,
            "Sharpe": sharpe
        })

###################################################################################################
# Benchmark Models (CAPM and FF3)
###################################################################################################
benchmarks = {
    "CAPM": fac[["Mkt-RF"]],
    "FF3" : fac[["Mkt-RF", "SMB", "RMW"]]
}

for name, f_df in benchmarks.items():
    print(f"Running OOS for benchmark: {name}")
    errors_b, preds_b, rstar_b = rolling_window_oos_given_factors(
        ret_q5, f_df, window=window
    )
    rms_b, ariv_b, sharpe_b = evaluate_oos(errors_b, preds_b, rstar_b)
    results.append({
        "Model": name,
        "Gamma": None,
        "k": f_df.shape[1],
        "RMS_alpha": rms_b,
        "ARIV": ariv_b,
        "Sharpe": sharpe_b
    })

###################################################################################################
# Combine and Display Final Table
###################################################################################################

results_df = pd.DataFrame(results)

results_df = results_df.sort_values(by=["Model", "Gamma", "k"]).reset_index(drop=True)

print("=== Final Q5 Out-of-Sample Results ===")
display(results_df)

